In [2]:
from mqt.bench import get_benchmark
from qiskit import QuantumCircuit, Aer, execute, assemble
from qiskit.providers.fake_provider import FakeSherbrooke
from qiskit.compiler import transpile
from qiskit.visualization import plot_gate_map
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
import time
import os
from statistics import mean
import matplotlib.pyplot as plt
import re
import csv
import warnings

# This bypasses the runtime warnings in the terminal
warnings.filterwarnings("ignore")

def sort_circuit(circuits):
    """ 
    Sorts a list of quantum circuits in descending order based on the number of qubits they contain.

    Parameters
    ----------
    circuits : list -> A list of quantum circuits to be sorted
    
    Returns
    -------
    None. -> To be called in the file reader to organize circuits to produce statistically significant graphs
    """

    for i in range(len(circuits)):
        min_qubits = circuits[i].num_qubits
        for j in range(len(circuits)):
            temp_qubits = circuits[j].num_qubits
            if min_qubits < temp_qubits:
                min_qubits = temp_qubits
                qc = circuits[i]
                circuits[i] = circuits[j]
                circuits[j] = qc

def file_reader(file_path):
    """ 
    This is the path to the quantum circuits stored from a specified directory and is responsible for 
    returning a list of QuantumCircuit objects along with a list of the order in which they were read.

    Parameters
    ----------
    file_path : str -> The path to the directory containing .qasm files.
    
    Returns
    -------
    circuits : tuple -> A list of QuantumCircuit objects read from the QASM files
    file_order : tuple -> A list of file names in the order they were read from the circuits in the first list.
    """  

    circuits = []
    file_order = []
    directory = file_path
    for circuit in os.listdir(directory):
        circuit_path = f"{file_path}/{circuit}"
        if(circuit_path.endswith('.qasm')):
            print(circuit_path)
            qc = QuantumCircuit.from_qasm_file(circuit_path)
            file_order.append(circuit)
            circuits.append(qc)
            
    #Sorts Circuits Before Transpiling:
    sort_circuit(circuits)
    return circuits, file_order

In [3]:
def runtime_benchmarking(NUM_ITERATIONS, circuits, the_backend):
    """ 
    This function benchmarks the runtime of transpilation for a list of quantum circuits on the different
    optimization levels (0, 1, 2, and 3). It calculates and displays the average runtime and creates
    a graph to visualize the runtime trends for each optimization level.

    Parameters
    ----------
    NUM_ITERATIONS : int -> The number of iterations for benchmarking each circuit
    circuits : list -> A list of quantum circuits we want to benchmark
    the_backend : backend -> The backend to used for transpilation

    Returns
    -------
    Optimization_Levels : tuple -> A dictionary of transpiled circuits grouped by optimization level.
    ttime_level1, ttime_level2, ttime_level3 : tuplw -> Lists of runtime data for each optimization level.
    mean_transpile_times_1, mean_transpile_times_2, mean_transpile_times_3: tuple -> Lists of mean transpile times for each optimization level.
    """

    BACKEND = the_backend
    Optimization_Levels = {3: [], 2: [], 1: [], 0: []}

    #These array will store:
    mean_transpile_times_1 = []
    mean_transpile_times_2 = []
    mean_transpile_times_3 = []
    
    ttime_level1 = []
    ttime_level2 = []
    ttime_level3 = []
    counter = 0
    
    for circuit in circuits:
        transpiled = False
        temp1 = []
        temp2 = []
        temp3 = []
        for _ in range(NUM_ITERATIONS):
            #Transpilation Level 1:
            start_time = time.perf_counter()
            qc1 = transpile(circuit, optimization_level= 1, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp1.append(stop_time - start_time)
            #Transpilation Level 2:
            start_time = time.perf_counter()
            qc2 = transpile(circuit, optimization_level= 2, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp2.append(stop_time - start_time)
            #Transpilation Level 3:
            start_time = time.perf_counter()
            qc3 = transpile(circuit, optimization_level= 3, seed_transpiler= 42, backend=BACKEND)
            stop_time = time.perf_counter()
            temp3.append(stop_time - start_time)
            #If this is the first iteration, then we simply add the circuits to the dictonary
            if transpiled == False:
                Optimization_Levels[3].append(qc3)
                Optimization_Levels[2].append(qc2)
                Optimization_Levels[1].append(qc1)
                Optimization_Levels[0].append(circuit)
                transpiled = True
        #At this point all the data has been added to iteration_times. Now it is just a matter of extracting data.
        ttime_level1.append(temp1)
        ttime_level2.append(temp2)
        ttime_level3.append(temp3)
        
        mean_transpile_times_1.append(mean(temp1))
        mean_transpile_times_2.append(mean(temp2))
        mean_transpile_times_3.append(mean(temp3))
        print("Circuit Index Completed: ", counter)
        counter += 1
    #Scatter Plot for Runtime after all values are collected
    plt.figure(figsize=(12, 6))
    #Number of qubits in the sorted circuit
    number_of_qubits = [i + 2 for i in range(len(mean_transpile_times_1))] #num of qubits
    x = np.array(number_of_qubits)
    #Calculating Line of BEST FIT: Optimization Level 1
    a, b = np.polyfit(x, np.array(mean_transpile_times_1), 1)
    #Calculating Line of BEST FIT: Optimization Level 2
    c, d = np.polyfit(x, np.array(mean_transpile_times_2), 1)
    #Calculating Line of BEST Fit: Optimization Level 3
    e, f = np.polyfit(x, np.array(mean_transpile_times_3), 1)
    plt.scatter(number_of_qubits, mean_transpile_times_1, label = "Average of Opt Level 1")
    plt.plot(x, a*x+b)
    plt.scatter(number_of_qubits, mean_transpile_times_2, label = "Average of Opt Level 2")
    plt.plot(x, c*x+d)
    plt.scatter(number_of_qubits, mean_transpile_times_3, label = "Average of Opt Level 3")
    plt.plot(x, e*x+f)
    plt.xlabel('Number of Qubits')
    plt.ylabel('Runtime in Seconds')
    plt.title('Runtime in Seconds (at each opt level)')
    plt.legend()
    plt.show()
    
    print("mean 1", mean_transpile_times_1)
    print("mean 2", mean_transpile_times_2)
    print("mean 3", mean_transpile_times_3)
    
    return Optimization_Levels, ttime_level1, ttime_level2, ttime_level3, mean_transpile_times_1, mean_transpile_times_2, mean_transpile_times_3

In [5]:
def gate_count(optimization_levels):
    """ 
    This function is responsible for counting and visualizing the number of gates for the transpiled 
    circuits at each optimization levels. It then returns the lists containing the gate counts 
    for each circuit at each optimization level.

    Parameters
    ----------
    optimization_levels : dict -> A dictionary containing the lists of transpiled circuits for each optimization level

    Returns
    -------
    opt1, opt2, opt3 : tuple -> A tuple containing lists of gate counts for each optimization level
    """ 

    # Transpile each circuit, count the gates, and store the results
    opt1 = []
    opt2 = []
    opt3 = []
    for i in range(len(optimization_levels[1])):
        # Counts the number of gates in a circuit given its respective optimization level
        opt1_count = optimization_levels[1][i].count_ops()
        opt2_count = optimization_levels[2][i].count_ops()
        opt3_count = optimization_levels[3][i].count_ops()

        # Appends our count of the transpiled circuit to the array
        opt1.append(sum(opt1_count.values()))
        opt2.append(sum(opt2_count.values()))
        opt3.append(sum(opt3_count.values()))

    number_of_qubits = [i + 2 for i in range(len(optimization_levels[1]))]   

    # Generate a graph to visualize the gate counts for each optimization levels
    plt.plot(number_of_qubits, opt1, label = "Optimization Level 1")
    plt.plot(number_of_qubits, opt2, label = "Optimization Level 2")
    plt.plot(number_of_qubits, opt3, label = "Optimization Level 3")
    plt.xlabel('Number of Qubits')
    plt.ylabel('Gate Count')
    plt.title('Gate Count of Transpiled Circuits')
    #plt.xticks(range(1, len(optimization_levels[1]) + 1))
    plt.legend()
    plt.show()

    print("gate count level 1", opt1)
    print("gate count level 2", opt2)
    print("gate count level 3", opt3)

    return opt1, opt2, opt3

In [7]:
backend = FakeSherbrooke() # Can change the backend you want to use
circuits, file_order = file_reader("/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19") # Can change the file name depending on the directory you want to use

# Retrieves the return values from the benchmarking methods for the CSV file
transpiled_circuits, level1_runtime, level2_runtime, level3_runtime, mean_transpile_times_1, mean_transpile_times_2, mean_transpile_times_3 = runtime_benchmarking(5, circuits, backend)
level1_gatecount, level2_gatecount, level3_gatecount = gate_count(transpiled_circuits)

/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_116.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_100.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_76.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_99.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_21.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_37.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_60.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_17.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_7.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_40.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-02-17-19/qft_indep_qiskit_56.qasm
/Users/guadalupecantera/Desktop/MQTBench_2023-08-04-0

In [ ]:
# Optimization Level 1 CSV
with open('optimzation_level_1.csv', 'w', newline='') as csvfile:
    # Below is the information we are trying to extract from our circuits
    fieldnames = ['Circuit Name',
                  ' Average Runtime: Level 1',
                  ' Run Times Level 1',
                  ' Gate Count: Level 1', 
                  ' Ratio: Level 1']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) # calling the writer
    writer.writeheader() # Here we are creating the columns for our CSV file

    for file_name, avg_lvl_1, runtime_lvl_1,gate_count_lvl_1, ratio_lvl_1 in zip(file_order, mean_transpile_times_1, level1_runtime, level1_gatecount, level1_ratio):
        writer.writerow({'Circuit Name': file_name,
                         ' Average Runtime: Level 1': avg_lvl_1,
                         ' Run Times Level 1': runtime_lvl_1, 
                         ' Gate Count: Level 1': gate_count_lvl_1, 
                         ' Ratio: Level 1': ratio_lvl_1 })

# Optimization Level 2 CSV
with open('optimzation_level_2.csv', 'w', newline='') as csvfile:
    # Below is the information we are trying to extract from our circuits
    fieldnames = ['Circuit Name',
                  ' Average Runtime: Level 2',
                  ' Run Times Level 2',
                  ' Gate Count: Level 2', 
                  ' Ratio: Level 2']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) # calling the writer
    writer.writeheader() # Here we are creating the columns for our CSV file

    for file_name, avg_lvl_2, runtime_lvl_2, gate_count_lvl_2, ratio_lvl_2 in zip(file_order, mean_transpile_times_2, level2_runtime, level2_gatecount, level2_ratio):
        writer.writerow({'Circuit Name': file_name,
                         ' Average Runtime: Level 2': avg_lvl_2,
                         ' Run Times Level 2': runtime_lvl_2, 
                         ' Gate Count: Level 2': gate_count_lvl_2, 
                         ' Ratio: Level 2': ratio_lvl_2 })
        
# Optimization Level 3 CSV        
with open('optimzation_level_3.csv', 'w', newline='') as csvfile:
    # Below is the information we are trying to extract from our circuits
    fieldnames = ['Circuit Name',
                  ' Average Runtime: Level 3',
                  ' Run Times Level 3',
                  ' Gate Count: Level 3', 
                  ' Ratio: Level 3']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames) # calling the writer
    writer.writeheader() # Here we are creating the columns for our CSV file

    for file_name, avg_lvl_3, runtime_lvl_3, gate_count_lvl_3, ratio_lvl_3 in zip(file_order, mean_transpile_times_3, level3_runtime, level3_gatecount, level3_ratio):
        writer.writerow({'Circuit Name': file_name,
                         ' Average Runtime: Level 3': avg_lvl_3,
                         ' Run Times Level 3': runtime_lvl_3, 
                         ' Gate Count: Level 3': gate_count_lvl_3, 
                         ' Ratio: Level 3': ratio_lvl_3 })